### Evaluation of criticality metrics on the nuscenes dataset.

**Results produced for the nuscenes dataset are needed, as for example they can be produced by the notebook MMDetection3D, see the other notebook MMDetection3D**

**Note: nuscenes-dev must be correctly installed** We used version 1.1.2, but it has been tested up to version 1.1.7.

#### Some hints on the modifications done to the nuscenes library:

- detection/algo.py : the method "accumulate" computes precision, recall and average precision. The approach to compute precision and recall is as follows. NPOS=TP+FN is a static value and can be computed immediately. Then, an array with all the detection thresholds for tps is computed e.g. $[1\; 0\; 0\; 0\; 1]$. Same is done for fp. Same is done with the detection confidence. Then they are sorted according to the detection confidence, and elements are picked up from top to bottom. Precision and recall are computed with the retrieved items, iteratively. This is the equivalent of starting getting "detected items" with the highest confidence, and progressively get all the remaining items, and compute precision and recall at each step with all the items selected so far. To compute the precision curve, there is a marvellous interpolation of the precision matrix and the recall matrix with a predefined matrix.


- detection/evaluate.py : contains the main method


- detection/data_classes.py : DetectionBox, in the init method, it includes all the logic to perform computation of all criticalities for a target BoundingBox i.e., for each object. This is done independently if it is a ground truth bounding box or a predicted bounding box. So whenever we plan to change our geometry, we need to act here.


Further, image creation is manipulated so that it shows criticality values matched to each bounding box that is depicted.  It visualizes only one type of objects (CAR), for clarity of the image --> to change this, in utils/data_classes.py, render_crit, change the line "if(self.name!='car'):"

Programmatically, the base is visualize_sample_crit in detection/render.py; then there is method render_crit; and finally we build object Box in utils/data_classes.py; images are created at the beginning of main in detection/evaluate.py; the label is added in render_crit in utils/data_classes.py. To print a different class (e.g., car, bus, everything), should be easy working with the above files (the no_crit case prints everything is on map).

**Debug images** Debug images (folder examples_debug_1) are introduced. They consider only ground truth, and they print: velocity and position of all vehicles,including the ego vehicle. They are created as follows:
- in detection/evaluate.py there is method visualize_sample_debug_1 of eval.detection.render
- in eval.detection.render method visualize_sample_debug_1, for each boxes in gt, there is render_debug_1 of utils/data_classes.py
- utils/data_classes.py method render_debug_1 builds the squares of the different ground truth cars

#### Which results are computed

Results are saved in different folders. They are easy to find, you just need to configure the proper path as it will be explained a bit later here.


We collect results in terms of:

- precision recall curve, that we save as pdf files (see the pdf files with "crit" in the name)

We compute some more metrics in a text file, that is called "confusion matrix.txt". The file contains: i) average precision; ii) average precision crit;

*Example: class_name bus; dist_th 1.0; ap 0.16393238098925264; ap_crit 0.16604208258718856*

iii) MAXIMUM number of tp and fp; iv) MAXIMUM value of tp_crit pred and MAXIMUM value of tp_crit gt; iii) MAXIMUM number of tp; iv) MAXIMUM value of fp_crit pred; v) MINIMUM number of FN, and MINIMUM value of fn_crit gt

*Example: class_name car; dist_th 0.5; max tp 19758.0; max fp 177143.0; min fn 33285.0; max tp_pred_crit 19249.891210080583; max tp_gt_crit 19272.72239514313; max fp_pred_crit 172455.20687223336; min fn_gt_crit 32468.569145629514*

To enrich this with multiple metrics, just operate on method *accumulate* in *algo.py*

Last, you can save bird eye view images that have the crit values added to each represented bounding box.

ACRONYMS (as naming in the following differs with respect to the paper)

First column is the algorithm name (as close as possible to naming in mmdetection3d), second is the abbreviation we use in our paper

- FCOSD-RESNET101              FCOS
- pointpillars-secfpn          SEC
- pointpillars-fpn             FPN
- regnet-regnetX_400MF-FPN     REG400
- ssn-SECFPN                   SSN
- regnet-regnetX_400MF-SECFPN  REGSEC (REG400SEC)
- ssn-REGNET                   SSNREG
- regnet-regnetX_FPN           REG1.6
- pgd                          PGD


Also, we have:

- D in the paper is:    $D_{max}$
- T in the paper is:        $T_{max}$
- I in the paper is:        $R_{max}$

*reminder:* if you ever need to change dist_ths, dist_thp: check configs/detection_cvpr_2019.json

Before this notebook is run, predictions must be made over the dataset for each of the detectors. Lets say the detector we want to run is the regnetX-1.6gf model. To run the detections on the GPU in mmdetection3d, run the following command from the mmdetection3d root directory: 


#### 1 pointpillars with backbone FPN (Feature Pyramid Networks) 
Model: https://download.openmmlab.com/mmdetection3d/v1.0.0_models/pointpillars/hv_pointpillars_fpn_sbn-all_4x8_2x_nus-3d/hv_pointpillars_fpn_sbn-all_4x8_2x_nus-3d_20210826_104936-fca299c1.pth

Results in: './pointpillars_nuscenes_results/pts_bbox/results_nusc.json' 

% ./tools/dist_test.sh configs/pointpillars/hv_pointpillars_fpn_sbn-all_4x8_2x_nus-3d.py checkpoints/hv_pointpillars_fpn_sbn-all_4x8_2x_nus-3d_20200620_230405-2fa62f3d.pth 2 --format-only --eval-options 'jsonfile_prefix=./results/pointpillars_nuscenes_results'

mAP: 39.7%
mAP: XX% (0.15 confidence filter)

#### 2 RegNET-X (RegNET WITH 400MF-FPN)
Model: https://github.com/open-mmlab/mmdetection3d/blob/master/configs/regnet/hv_pointpillars_regnet-400mf_fpn_sbn-all_4x8_2x_nus-3d.py

Results in: './regnetX_nuscenes_results/pts_bbox/results_nusc.json'

% ./tools/dist_test.sh configs/regnet/hv_pointpillars_regnet-400mf_fpn_sbn-all_4x8_2x_nus-3d.py checkpoints/hv_pointpillars_regnet-400mf_fpn_sbn-all_4x8_2x_nus-3d_20200620_230239-c694dce7.pth 2 --format-only --eval-options 'jsonfile_prefix=./results/regnetX_nuscenes_results'

mAP: 44.84%
mAP: XX% (0.15 confidence filter)

#### 3 SSN with backbone REGNET 
Model: https://github.com/open-mmlab/mmdetection3d/blob/master/configs/ssn/hv_ssn_regnet-400mf_secfpn_sbn-all_2x16_2x_nus-3d.py

Results in: './ssn_nuscenes_results/pts_bbox/results_nusc-regnet.json'

% ./tools/dist_test.sh configs/ssn/hv_ssn_regnet-400mf_secfpn_sbn-all_2x16_2x_nus-3d.py checkpoints/hv_ssn_regnet-400mf_secfpn_sbn-all_2x16_2x_nus-3d_20201024_232447-7af3d8c8.pth 2 --format-only --eval-options 'jsonfile_prefix=./results/ssn_nuscenes_results'

mAP:46.65%
mAP: XX% (0.15 confidence filter)



#### Below are configuration items

In [1]:


## POINTPILLARS WITH FPN BACKBONE -- lidar only
#PATH='/cluster/work/andronn/MasterThesis/MASTER/mmdetection3d/results/pointpillars_nuscenes_results/pts_bbox/'
#FILE_JSON='results_nusc.json'
#DETECTOR='pointpillars-fpn'

##RegNET WITH BACKBONE RegNetX-1.6gF-FPN
#PATH='/cluster/work/andronn/MasterThesis/MASTER/mmdetection3d/results/regnetX_nuscenes_results/pts_bbox/' 
#FILE_JSON='results_nusc.json'
#DETECTOR='regnet-regnetX_FPN'

## SSN con backbone REGNET 400MF SECFPN  -- LIDAR-only
PATH='/cluster/work/andronn/MasterThesis/MASTER/mmdetection3d/results/ssn_nuscenes_results/pts_bbox/' 
FILE_JSON='results_nusc.json'
DETECTOR='ssn-REGNET'

#nuscene dataroot
DATAROOT = '/cluster/work/andronn/MasterThesis/MASTER/mmdetection3d/data/nuscenes'
#path + json file where detection results from mmdetection3d are stored, ready to be processed

RESULT_PATH=PATH+FILE_JSON
#the detector in use. Does nothing special but creates a folder, and it is useful to put results there
#results of evaluation will be stored here
OUTPUT='/cluster/work/andronn/MasterThesis/MASTER/master_repo/Thesis-Evaluating-Safety-Oriented-Metrics-for-Object-Detectors/results/'+DETECTOR+"/"

#parameters from our marvellous solution. At the bottom of the notebook, there is a different 
#approach where multiple config are tested
MAX_DISTANCE_OBJ_1=30.0
MAX_DISTANCE_INTERSECT_1=20.0
MAX_TIME_INTERSECT_OBJ_1=10.0

#how many images in bird view you want to draw
NUMBER_IMAGE=3

In [2]:
#required if you are playing with libraries and changing them. This reloads libraries
%load_ext autoreload
%autoreload 2

In [3]:
import unittest
import numpy as np
import sklearn
import tqdm
import pandas
import math
import json
from typing import Callable
from nuscenes import NuScenes
from nuscenes.eval.prediction.splits import *
import nuscenes.eval.detection.config as cnfig
from nuscenes.eval.detection.configs import *
from nuscenes.eval.detection.data_classes import DetectionBox 
from nuscenes.eval.detection import *
import nuscenes.eval.detection.algo as ag
from nuscenes.eval.detection.data_classes import DetectionMetricData, DetectionConfig, DetectionMetrics, DetectionBox, \
    DetectionMetricDataList
from nuscenes.eval.common.data_classes import EvalBoxes

import os
from typing import List, Dict, Callable, Tuple
from nuscenes.eval.common.utils import center_distance, scale_iou, yaw_diff, velocity_l2, attr_acc, cummean
import nuscenes.eval.detection.evaluate as dcl    
from nuscenes.prediction import *
from nuscenes.map_expansion.map_api import *

In [ ]:
nuscenes = NuScenes('v1.0-trainval', dataroot=DATAROOT)
confvalue=cnfig.config_factory("detection_cvpr_2019")

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 49.258 seconds.
Reverse indexing ...


In [ ]:
#number of scene that compose the val set
eval=val = \
    ['scene-0003', 'scene-0012', 'scene-0013', 'scene-0014', 'scene-0015', 'scene-0016', 'scene-0017', 'scene-0018',
     'scene-0035', 'scene-0036', 'scene-0038', 'scene-0039', 'scene-0092', 'scene-0093', 'scene-0094', 'scene-0095',
     'scene-0096', 'scene-0097', 'scene-0098', 'scene-0099', 'scene-0100', 'scene-0101', 'scene-0102', 'scene-0103',
     'scene-0104', 'scene-0105', 'scene-0106', 'scene-0107', 'scene-0108', 'scene-0109', 'scene-0110', 'scene-0221',
     'scene-0268', 'scene-0269', 'scene-0270', 'scene-0271', 'scene-0272', 'scene-0273', 'scene-0274', 'scene-0275',
     'scene-0276', 'scene-0277', 'scene-0278', 'scene-0329', 'scene-0330', 'scene-0331', 'scene-0332', 'scene-0344',
     'scene-0345', 'scene-0346', 'scene-0519', 'scene-0520', 'scene-0521', 'scene-0522', 'scene-0523', 'scene-0524',
     'scene-0552', 'scene-0553', 'scene-0554', 'scene-0555', 'scene-0556', 'scene-0557', 'scene-0558', 'scene-0559',
     'scene-0560', 'scene-0561', 'scene-0562', 'scene-0563', 'scene-0564', 'scene-0565', 'scene-0625', 'scene-0626',
     'scene-0627', 'scene-0629', 'scene-0630', 'scene-0632', 'scene-0633', 'scene-0634', 'scene-0635', 'scene-0636',
     'scene-0637', 'scene-0638', 'scene-0770', 'scene-0771', 'scene-0775', 'scene-0777', 'scene-0778', 'scene-0780',
     'scene-0781', 'scene-0782', 'scene-0783', 'scene-0784', 'scene-0794', 'scene-0795', 'scene-0796', 'scene-0797',
     'scene-0798', 'scene-0799', 'scene-0800', 'scene-0802', 'scene-0904', 'scene-0905', 'scene-0906', 'scene-0907',
     'scene-0908', 'scene-0909', 'scene-0910', 'scene-0911', 'scene-0912', 'scene-0913', 'scene-0914', 'scene-0915',
     'scene-0916', 'scene-0917', 'scene-0919', 'scene-0920', 'scene-0921', 'scene-0922', 'scene-0923', 'scene-0924',
     'scene-0925', 'scene-0926', 'scene-0927', 'scene-0928', 'scene-0929', 'scene-0930', 'scene-0931', 'scene-0962',
     'scene-0963', 'scene-0966', 'scene-0967', 'scene-0968', 'scene-0969', 'scene-0971', 'scene-0972', 'scene-1059',
     'scene-1060', 'scene-1061', 'scene-1062', 'scene-1063', 'scene-1064', 'scene-1065', 'scene-1066', 'scene-1067',
     'scene-1068', 'scene-1069', 'scene-1070', 'scene-1071', 'scene-1072', 'scene-1073']

In [ ]:
len(val)

In [ ]:
%reload_ext autoreload

dt=dcl.DetectionEval(nuscenes,confvalue, RESULT_PATH, 'val', OUTPUT, verbose=False, MAX_DISTANCE_OBJ=MAX_DISTANCE_OBJ_1,
                     MAX_DISTANCE_INTERSECT=MAX_DISTANCE_INTERSECT_1, MAX_TIME_INTERSECT_OBJ=MAX_TIME_INTERSECT_OBJ_1)

In [ ]:
modified_predictions = False
# Add False Positives and False Negatives to detection results
if modified_predictions==True:
    dt.add_FN('28cce0b8fdb7437db096beb047179453', 20) # Remove truck in front of vehicle in sample
    dt.add_FN('645e0c2f9b724c5b95fe434930c2682a', 20) # Remove bus in front of vehicle in sample
    dt.add_FP('6c8d4379e83646d08436f6ec92b35fe5', (3,5), (1.7, 4.0, 2.0)) # Add FP in front of ego
    dt.add_FP('970e605b05c14027b08a6e74d5e8ceca', (0,10), (1.7, 4.0, 2.0), True) # FP 10m in front of ego
    dt.add_FP('b8849fcd11d14e499aea2ee258d8b581', (5,0), (1.7, 4.0, 2.0), True)
    dt.add_FN('c35efe5eaf784275b8c7e31fb50aa902', 20) # FNs for pedestrian detections
    dt.add_FN('c752fd12565d48d598a8bfce5b0530ad', 10) # FNs for closest vehicles
    dt.add_FP('d056b9bdd56f44669540c6c323042d30', (0,15), (1.7, 4.0, 2.0)) # FP 15m ahead of ego, STOPPED
    dt.add_FN('d575e4dbdd6749068d4e71f7d99b02bc', 37) # FN for single vehicle detection
    dt.add_FP('ec78e68b1637464da5305a9cbed214c4', (-3,12), (1.7, 4.0, 2.0), True) # FP in opposite lane matching ego speed
    dt.add_FN('f4a78e352cb74deca0bf4e1669aec42a', 20, False) # Remove one! of the detections in front of ego 


In [ ]:
%reload_ext autoreload
ex_tokens_path='/cluster/work/andronn/MasterThesis/MASTER/master_repo/Thesis-Evaluating-Safety-Oriented-Metrics-for-Object-Detectors/results/sample_tokens_selected.json'

with open(ex_tokens_path, 'r') as f:
    sample_tokens_qualitative = list(json.load(f)['sample_tokens'])
dt.main(plot_examples=NUMBER_IMAGE,
        render_curves=True, 
        model_name=DETECTOR,
        MAX_DISTANCE_OBJ=MAX_DISTANCE_OBJ_1,
        MAX_DISTANCE_INTERSECT=MAX_DISTANCE_INTERSECT_1,
        MAX_TIME_INTERSECT=MAX_TIME_INTERSECT_OBJ_1,
        recall_type="PRED AL NUMERATORE",
        save_metrics_samples=True,
        single_sample_tokens=sample_tokens_qualitative,
        modified_predictions=modified_predictions) #This must be "PRED AL NUMERATORE", to match results of the paper


In [17]:
# ALTERNATIVE CELL FOR GENERATING SINGLE DETECTOR METRIC RESULTS ON LARGER RANDOM SAMPLES

%reload_ext autoreload
results_path='/cluster/work/andronn/MasterThesis/MASTER/master_repo/Thesis-Evaluating-Safety-Oriented-Metrics-for-Object-Detectors/results/'

random_tokens_path = os.path.join(results_path, 'sample_tokens_random.json')
with open(random_tokens_path, 'r') as f:
    sample_tokens = list(json.load(f)['sample_tokens'])[:100]
    
random_modified_preds = False
CONF_TH = 0.40
classes = ['car'] # classes to remove when inserting FNs
num_falses = random.randint(0, 4)
#dist = 40
if random_modified_preds:
    # add set or random FPs to all samples
    
    
    import random
    for token in sample_tokens: 
        
        remove_all = random.getrandbits(1)&random.getrandbits(1) # 1/4 prob
        
        for i in range(num_falses):
            x = random.uniform(-5,5)
            y = random.uniform(-10, 30)
            dist = random.uniform(10, 30) if remove_all else random.uniform(10, 40)
            randsize = (random.uniform(1.5, 3.0), random.uniform(3.0, 6.0), random.uniform(1.5, 3.5))
            dt.add_FP(token, (x, y), randsize, random.getrandbits(1)) 
            #dt.add_FN(sample_token=token, dist=dist, remove_all=remove_all, classes=classes)
            if remove_all: break

        
    
dt.main(plot_examples=NUMBER_IMAGE,
        render_curves=False, 
        model_name=DETECTOR,
        MAX_DISTANCE_OBJ=MAX_DISTANCE_OBJ_1,
        MAX_DISTANCE_INTERSECT=MAX_DISTANCE_INTERSECT_1,
        MAX_TIME_INTERSECT=MAX_TIME_INTERSECT_OBJ_1,
        recall_type="PRED AL NUMERATORE",
        save_metrics_samples=True,
        single_sample_tokens=sample_tokens,
        modified_predictions=False,
        random_token_predictions=True,
        conf_th_sample=CONF_TH) #This must be "PRED AL NUMERATORE", to match results of the paper



STARTING EVALUATION in main (self)
Made sample directory: /cluster/work/andronn/MasterThesis/MASTER/master_repo/Thesis-Evaluating-Safety-Oriented-Metrics-for-Object-Detectors/results/ssn-REGNET/METRIC_SAMPLES_RANDOM/2be58f6df6d745438fe056ae74c82228



/cluster/home/andronn/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/cluster/home/andronn/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


using model weights ./planner.pt
using location masks ./masks_trainval.json
calculating pkl...


100%|████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.84s/it]


plotting 0 timestamps...
Saved metric data for sample 2be58f6df6d745438fe056ae74c82228
Sample 2be58f6df6d745438fe056ae74c82228 data saved.

Made sample directory: /cluster/work/andronn/MasterThesis/MASTER/master_repo/Thesis-Evaluating-Safety-Oriented-Metrics-for-Object-Detectors/results/ssn-REGNET/METRIC_SAMPLES_RANDOM/a6c2ae7542054cf19a68bd7bfc3dc478

using model weights ./planner.pt
using location masks ./masks_trainval.json
calculating pkl...


...

plotting 0 timestamps...
Saved metric data for sample 4967dbb7fa484682ad75504a986b94b1
Sample 4967dbb7fa484682ad75504a986b94b1 data saved.

Made sample directory: /cluster/work/andronn/MasterThesis/MASTER/master_repo/Thesis-Evaluating-Safety-Oriented-Metrics-for-Object-Detectors/results/ssn-REGNET/METRIC_SAMPLES_RANDOM/e1dffaba060040cfab07dec04790fbfa

using model weights ./planner.pt
using location masks ./masks_trainval.json
calculating pkl...


100%|████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.45s/it]


plotting 0 timestamps...
Saved metric data for sample e1dffaba060040cfab07dec04790fbfa
Sample e1dffaba060040cfab07dec04790fbfa data saved.

Made sample directory: /cluster/work/andronn/MasterThesis/MASTER/master_repo/Thesis-Evaluating-Safety-Oriented-Metrics-for-Object-Detectors/results/ssn-REGNET/METRIC_SAMPLES_RANDOM/e553a1dfab23413c848d30b585181fe8

using model weights ./planner.pt
using location masks ./masks_trainval.json
calculating pkl...


100%|████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.78s/it]


plotting 0 timestamps...
Saved metric data for sample e553a1dfab23413c848d30b585181fe8
Sample e553a1dfab23413c848d30b585181fe8 data saved.

Rendering sample token 128314940251481d955daa53592444ac
Rendering sample token 697cc6a77d9841209fdea6458fe3be51
Rendering sample token 4e87276d0d1347a48550a74799631d6a
Rendering sample token 128314940251481d955daa53592444ac
Rendering sample token 697cc6a77d9841209fdea6458fe3be51
Rendering sample token 4e87276d0d1347a48550a74799631d6a
Rendering sample token 128314940251481d955daa53592444ac
Rendering sample token 697cc6a77d9841209fdea6458fe3be51
Rendering sample token 4e87276d0d1347a48550a74799631d6a
Rendering sample token 128314940251481d955daa53592444ac
Rendering sample token 697cc6a77d9841209fdea6458fe3be51
Rendering sample token 4e87276d0d1347a48550a74799631d6a
Rendering sample token 128314940251481d955daa53592444ac
Rendering sample token 697cc6a77d9841209fdea6458fe3be51
Rendering sample token 4e87276d0d1347a48550a74799631d6a


### Calculate PKLs
Using planning_centric_metrics, calculate PKL for dataset (deprecated, PKL is calculated in calc_sample_crit for each sample for shorter computation)

In [ ]:
"""dt.filter_boxes_confidence(conf_th=0.15) # Filter BBs on confidence before PKL evaluation. default = 0.15

from nuscenes.map_expansion.map_api import NuScenesMap
from planning_centric_metrics import calculate_pkl
import torch

print(torch.cuda.device_count())
print(torch.cuda.is_available())
gpuid = -1
device = torch.device(f'cuda:{gpuid}') if gpuid >= 0\
        else torch.device('cpu')
print('using device: {0}'.format(device))

map_folder = '/cluster/work/andronn/MasterThesis/MASTER/mmdetection3d/data/nuscenes/maps/'
nusc_maps = {map_name: NuScenesMap(dataroot=map_folder,
                 map_name=map_name) for map_name in [
                    "singapore-hollandvillage",
                    "singapore-queenstown",
                    "boston-seaport",
                    "singapore-onenorth",
                ]}
nworkers = 2

pkl = calculate_pkl(dt.gt_boxes, dt.pred_boxes,
                         dt.sample_tokens, dt.nusc,
                         nusc_maps, device,
                         nworkers=nworkers, bsz=16,
                         plot_kextremes=0,
                         verbose=True)
with open(os.path.join(OUTPUT,'pkl_results.json'), 'w') as fp:
    json.dump(pkl, fp)
"""